In [1]:
import os
os.chdir('../')

In [2]:
import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
#from collections import Counter

In [3]:
train_df = data.train_df('small')

In [18]:
train_df.head(15)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1


Average sessions length

### Sessions length

In [15]:
train_df[['session_id', 'action_type']].groupby('session_id').count()['action_type'].describe()

count    14009.000000
mean         5.680705
std         10.557188
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max        751.000000
Name: action_type, dtype: float64

## Analyze the sessions with at least 1 clickout

In [6]:
def count_interactions_after_clickout(g):
    clickouts = g[g.action_type == 'clickout item']
    if len(clickouts) > 0:
        g = g.reset_index()
        last_click_index = g[ g['index'] == clickouts.iloc[[-1]].index.values[0] ].index.values[0]
        return pd.Series({'clickout_reverse_index': len(g) - last_click_index})

In [12]:
t = train_df.groupby('session_id').progress_apply(count_interactions_after_clickout)
t = t[t.clickout_reverse_index.notnull()]
t

In [13]:
t

,clickout_reverse_index
session_id,
0008f9d105bbf,2.0
000c9723c8de0,1.0
000d3192b88d7,2.0
000ef61e73aef,1.0
001194d0b4d55,1.0
0012e067b9cfc,2.0
0018c0eba0994,1.0
002e23d051bf6,1.0
002e66d4a7aff,1.0


### Average reverse index

In [10]:
t.clickout_reverse_index.describe()

count    12719.000000
mean         1.803129
std          5.266905
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        470.000000
Name: clickout_reverse_index, dtype: float64

In [14]:
t.sort_values('clickout_reverse_index', ascending=False)

,clickout_reverse_index
session_id,
948641e533837,470.0
a1c1b7a2f1eb8,97.0
800a6688cb1ae,88.0
c8236e8117f6c,83.0
dc5a2c8088144,65.0
2af1480f6a651,62.0
88d004b31d9ee,59.0
60aaf73f93575,59.0
cedc9905ec596,58.0


In [17]:
train_df[train_df.session_id == '0042c51d0af67']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
47193,W4E5BBYN4KMT,0042c51d0af67,1541379916,1,clickout item,104734,BR,"Goiânia, Brazil",mobile,NaN,104734|7195486|5862328|1153302|104942|926265|5...,66|46|64|42|72|56|53|43|51|44|52|55|37|70|59|3...,1
47194,W4E5BBYN4KMT,0042c51d0af67,1541379922,3,interaction item image,104734,BR,"Goiânia, Brazil",mobile,NaN,NaN,NaN,2
47195,W4E5BBYN4KMT,0042c51d0af67,1541379922,4,interaction item info,104734,BR,"Goiânia, Brazil",mobile,NaN,NaN,NaN,1


In [18]:
train_df[train_df.session_id == '00520d47dc541']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
88189,40HINRQEEUI1,00520d47dc541,1541265362,1,interaction item rating,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,1
88190,40HINRQEEUI1,00520d47dc541,1541265469,2,interaction item info,791286,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,1
88191,40HINRQEEUI1,00520d47dc541,1541265522,3,clickout item,791286,NO,"Smögen, Sweden",mobile,NaN,7709608|738401|1710563|2865068|8191686|8423470...,116|193|164|48|67|73|113|115|96|106|107|184,1
88192,40HINRQEEUI1,00520d47dc541,1541270196,6,interaction item info,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,3
88193,40HINRQEEUI1,00520d47dc541,1541270244,7,clickout item,7709608,NO,"Smögen, Sweden",mobile,NaN,738401|7709608|2376022|1710563|1991305|2865068...,164|116|116|145|87|48|125|67|68|135|73|115|101...,1
88194,40HINRQEEUI1,00520d47dc541,1541270665,9,interaction item image,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,2
88195,40HINRQEEUI1,00520d47dc541,1541270716,11,interaction item info,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,2
88196,40HINRQEEUI1,00520d47dc541,1541270730,12,interaction item rating,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,1
88197,40HINRQEEUI1,00520d47dc541,1541270764,14,interaction item info,7709608,NO,"Smögen, Sweden",mobile,NaN,NaN,NaN,2
88198,40HINRQEEUI1,00520d47dc541,1541270767,15,clickout item,7709608,NO,"Smögen, Sweden",mobile,NaN,738401|7709608|2376022|1710563|1991305|2865068...,164|116|116|145|87|48|125|67|68|135|73|115|101...,1


In [30]:
train_df[train_df.session_id == 'ff97eb096d9ba']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
41448,MWLV35PG3CCN,ff97eb096d9ba,1541342604,2,search for destination,"Manzanillo, Mexico",MX,"Manzanillo, Mexico",mobile,NaN,NaN,NaN,2
41449,MWLV35PG3CCN,ff97eb096d9ba,1541342624,3,clickout item,43030,MX,"Manzanillo, Mexico",mobile,NaN,42996|43030|437751|944763|34295|370256|1333662...,140|84|101|54|119|60|47|47|26|24|31|101|72|104...,1
41450,MWLV35PG3CCN,ff97eb096d9ba,1541342800,26,interaction item image,370256,MX,"Manzanillo, Mexico",mobile,NaN,NaN,NaN,23
41451,MWLV35PG3CCN,ff97eb096d9ba,1541342877,27,interaction item info,42406,MX,"Manzanillo, Mexico",mobile,NaN,NaN,NaN,1
41452,MWLV35PG3CCN,ff97eb096d9ba,1541343121,29,clickout item,42406,MX,"Manzanillo, Mexico",mobile,NaN,42406|42996|43030|437751|34295|944763|42787|37...,38|161|106|116|122|72|34|92|33|26|47|51|42|47|...,2
41453,MWLV35PG3CCN,ff97eb096d9ba,1541344486,30,clickout item,2054338,MX,"Manzanillo, Mexico",mobile,NaN,42996|43030|437751|944763|34295|370256|1333662...,140|84|101|54|119|60|47|47|26|24|31|101|104|72...,1
41454,MWLV35PG3CCN,ff97eb096d9ba,1541344601,34,interaction item image,2054338,MX,"Manzanillo, Mexico",mobile,NaN,NaN,NaN,4


In [29]:
train_df[train_df.city == 'Maragogi, Brazil']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
931,5FNP9AOWO3MO,9cc22c7accd8b,1541103151,1,search for item,421381,BR,"Maragogi, Brazil",desktop,NaN,NaN,NaN,1
932,5FNP9AOWO3MO,9cc22c7accd8b,1541103182,2,clickout item,421381,BR,"Maragogi, Brazil",desktop,NaN,421381|503201|2710052|1668489|5965972|5427540|...,354|185|36|67|81|36|122|97|65|91|78|285|68|27|...,1
1779,C95T1FZM28CS,fd8f2985074a2,1541074705,1,search for destination,"Maragogi, Brazil",AR,"Maragogi, Brazil",desktop,NaN,NaN,NaN,1
1780,C95T1FZM28CS,fd8f2985074a2,1541074742,2,clickout item,3094482,AR,"Maragogi, Brazil",desktop,NaN,3094482|2710052|421381|503201|1776249|9758640|...,177|59|284|223|153|41|29|129|101|88|67|79|55|4...,1
8964,1FYTGBYD86G1,6d685757d1a62,1541206873,1,clickout item,3094482,BR,"Maragogi, Brazil",tablet,NaN,3094482|503201|1776249|2356278|1677213|2571728...,347|310|249|134|209|87|107|102|106|130|147|158...,1
10703,I19SFI04UE3L,34f6a41931a82,1541266840,1,search for destination,"Maragogi, Brazil",BR,"Maragogi, Brazil",mobile,NaN,NaN,NaN,1
13541,5JVN68PEDGDK,17ab2e5212415,1541373805,1,search for destination,"Maragogi, Brazil",AR,"Maragogi, Brazil",desktop,NaN,NaN,NaN,1
13542,5JVN68PEDGDK,17ab2e5212415,1541373865,2,clickout item,2249828,AR,"Maragogi, Brazil",desktop,NaN,503201|1776249|2717947|2363822|2673221|1776247...,295|227|218|136|85|165|145|103|118|59|212|122|...,1
27589,9BZGAF7SUJTL,a86de3b29bc4c,1541031381,14,interaction item image,2757522,BR,"Maragogi, Brazil",mobile,NaN,NaN,NaN,14
27590,9BZGAF7SUJTL,a86de3b29bc4c,1541031381,15,clickout item,2757522,BR,"Maragogi, Brazil",mobile,NaN,2757522|2710052|5427540|2363822|2680358|166848...,70|38|41|50|50|92|172|97|23|50|45|34|51|50|102...,1


In [45]:
train = data.train_df('small', 'cluster_recurrent')
#test_df = data.test_df('small', 'cluster_recurrent')

In [49]:
train[(train.user_id == 'C2D58CUH9AOT') & (train.session_id == '000263df674fa')]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
284016,C2D58CUH9AOT,000263df674fa,1541449907,2,search for destination,"Orlando, USA",CL,"Orlando, USA",desktop,NaN,NaN,NaN,2
284017,C2D58CUH9AOT,000263df674fa,1541449928,3,search for item,8249,CL,"Lake Buena Vista, USA",desktop,NaN,NaN,NaN,1
284018,C2D58CUH9AOT,000263df674fa,1541449962,4,clickout item,8249,CL,"Lake Buena Vista, USA",desktop,NaN,8249|8250|14378|67361|8284|8257|1955219|8254|6...,74|88|87|72|56|87|244|168|123|50|129|72|151|83...,1
284019,C2D58CUH9AOT,000263df674fa,1541450198,5,clickout item,8249,CL,"Lake Buena Vista, USA",desktop,NaN,8249|8250|14378|67361|8284|8257|1955219|8254|6...,74|88|87|72|56|87|244|168|123|50|129|72|151|83...,1


In [10]:
pop = pd.read_csv('dataset/preprocessed/no_cluster/full/feature/reference_position_in_next_clickout_impressions/features.csv', index_col=0)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
pop.loc[933:950]

,user_id,session_id,ref_pos
933,5BR973YOX5QD,a24a3d2fb4543,2
934,5BR973YOX5QD,a24a3d2fb4543,2
935,5BR973YOX5QD,a24a3d2fb4543,2
936,5BR973YOX5QD,a24a3d2fb4543,1
937,5BR973YOX5QD,a24a3d2fb4543,3
938,5BR973YOX5QD,a24a3d2fb4543,21
939,5BR973YOX5QD,a24a3d2fb4543,0
940,5BR973YOX5QD,a24a3d2fb4543,3
941,5BR973YOX5QD,a24a3d2fb4543,21
943,5C84D2WKQYX0,f463d9661fdac,0


In [16]:
train_df.loc[933:950]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
933,5BR973YOX5QD,a24a3d2fb4543,1541080749,59,search for item,1496061,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,1
934,5BR973YOX5QD,a24a3d2fb4543,1541080816,91,interaction item image,1496061,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,32
935,5BR973YOX5QD,a24a3d2fb4543,1541080816,92,interaction item rating,1496061,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,1
936,5BR973YOX5QD,a24a3d2fb4543,1541080840,100,interaction item image,479176,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,8
937,5BR973YOX5QD,a24a3d2fb4543,1541081073,103,interaction item image,1708467,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,3
938,5BR973YOX5QD,a24a3d2fb4543,1541081090,111,interaction item image,1627077,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,8
939,5BR973YOX5QD,a24a3d2fb4543,1541081093,112,interaction item image,513366,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,1
940,5BR973YOX5QD,a24a3d2fb4543,1541081103,113,interaction item image,1708467,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,1
941,5BR973YOX5QD,a24a3d2fb4543,1541081103,114,interaction item image,1627077,BR,"Praia da Pipa, Brazil",mobile,NaN,NaN,NaN,1
942,5BR973YOX5QD,a24a3d2fb4543,1541081113,115,clickout item,1496061,BR,"Praia da Pipa, Brazil",mobile,NaN,513366|479176|1496061|1708467|8803444|1963275|...,46|49|46|16|40|46|48|41|45|146|59|154|122|236|...,1


In [19]:
price = pd.read_csv('dataset/preprocessed/no_cluster/full/feature/reference_price_in_next_clickout/features.csv', index_col=0)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
price.loc[933:950]

,user_id,session_id,price
933,5BR973YOX5QD,a24a3d2fb4543,46
934,5BR973YOX5QD,a24a3d2fb4543,46
935,5BR973YOX5QD,a24a3d2fb4543,46
936,5BR973YOX5QD,a24a3d2fb4543,49
937,5BR973YOX5QD,a24a3d2fb4543,16
938,5BR973YOX5QD,a24a3d2fb4543,48
939,5BR973YOX5QD,a24a3d2fb4543,46
940,5BR973YOX5QD,a24a3d2fb4543,16
941,5BR973YOX5QD,a24a3d2fb4543,48
943,5C84D2WKQYX0,f463d9661fdac,90
